In [71]:
import pandas as pd
import os
import glob
import numpy as np

In [26]:
#On fixe le repertoire de travail
os.chdir(r'C:\Users\106443\ML_Test\batchs')

#Importer tous les fichiers présent dans le repertoire 
extension = 'csv'
all_filenames =[ i for i in glob.glob('*.{}'.format(extension))]

df = pd.concat([pd.read_csv(f) for f in all_filenames])

In [52]:
#Importation fichier : méthode du prof
start_week = 61
end_week = 70
prefix = 'restaurant_1'
data_dir = 'C:/Users/106443/ML_Test'

In [108]:
#Data directory
def extract(data_dir: str, prefix: str, start_week: int, end_week: int) -> pd.DataFrame:
    """ Extract a temporal slice of data (from start week to end week)
    
    Parameters
    ----------
    data_dir: str
        Data directory path.
    prefix: str
        Data source identidication (e.g. restautant_1)
    start_week: int
        First week number (inclued)
    end_week: int
        Last wee number (inclued).
    
    Return
    ------
    pd.DataFrame
        Temporal slide of data
    """
    df = pd.DataFrame()
    for i in range(start_week,end_week+1):
        file_path = os.path.join(data_dir, 'batchs',f'{prefix}_week_{i}.csv')
        if os.path.isfile(file_path): #Si le chemin existe
            batch = pd.read_csv(file_path)
            print()
            df = pd.concat([df,batch],sort =True)
    return df


In [109]:
df = extract('C:/Users/106443/ML_Test', 'restaurant_1', 134,174)
df

,Item Name,Order Date,Order Number,Product Price,Quantity,Total products
0,Red Sauce,2017-07-30 19:52:00,6235,0.50,1,9
1,Mango Chutney,2017-07-30 19:28:00,6233,0.50,2,5
2,Mango Chutney,2017-07-30 16:10:00,6226,0.50,1,8
3,Mango Chutney,2017-07-29 19:44:00,6218,0.50,1,9
4,Mango Chutney,2017-07-29 17:20:00,6210,0.50,1,10
...,...,...,...,...,...,...
592,Bhuna,2018-05-03 19:01:00,8752,12.95,1,8
593,King Prawn Shaslick,2018-05-01 22:25:00,8081,12.95,1,6
594,Hazary Lamb,2018-05-01 18:29:00,8069,12.95,2,18
595,House Red wine 75cl,2018-05-05 13:14:00,9415,17.95,1,9


In [110]:
#On renome les colonnes
df.columns = df.columns.str.lower().str.replace(' ','_')

In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17871 entries, 0 to 596
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   item_name       17871 non-null  object 
 1   order_date      17871 non-null  object 
 2   order_number    17871 non-null  int64  
 3   product_price   17871 non-null  float64
 4   quantity        17871 non-null  int64  
 5   total_products  17871 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 977.3+ KB


In [112]:
#On compte les valeurs manquantes
for col in list(df.columns):
    print('la variable ' + col + ' a ' + str(df[col].isna().sum()) + ' valeurs manquantes '+str(round(df[col].isna().sum()/len(df)*100,2))+'%')

la variable item_name a 0 valeurs manquantes 0.0%
la variable order_date a 0 valeurs manquantes 0.0%
la variable order_number a 0 valeurs manquantes 0.0%
la variable product_price a 0 valeurs manquantes 0.0%
la variable quantity a 0 valeurs manquantes 0.0%
la variable total_products a 0 valeurs manquantes 0.0%


In [113]:
#CA
df['ca'] = df['product_price'] * df['quantity']

In [114]:
#CA par Order number et Order date
df['cash_in'] = df.groupby(['order_number'])['ca'].transform(np.sum)

In [115]:
#On supprime les variables inutiles
df = df.drop(columns= ['item_name','quantity','product_price','total_products','ca'])

In [116]:
#On supprime les doublons
df = df.drop_duplicates()

In [117]:
#On remet les index correctement
df = df.reset_index(drop =True)

In [118]:
#Penser à mettre toutes les étapes de nettoyage dans une fonction !!!

In [119]:
df.head()

,order_date,order_number,cash_in
0,2017-07-30 19:52:00,6235,41.75
1,2017-07-30 19:28:00,6233,26.40
2,2017-07-30 16:10:00,6226,37.25
3,2017-07-29 19:44:00,6218,53.05
4,2017-07-29 17:20:00,6210,64.70


In [120]:
df['order_date'] = pd.to_datetime(df['order_date'])
df.resample('1H', on='order_date').sum().reset_index()


,order_date,order_number,cash_in
0,2017-01-01 17:00:00,12462,234.05
1,2017-01-01 18:00:00,24951,184.70
2,2017-01-01 19:00:00,4162,21.05
3,2017-01-01 20:00:00,4163,27.80
4,2017-01-01 21:00:00,0,0.00
...,...,...,...
17496,2018-12-31 17:00:00,76437,259.85
17497,2018-12-31 18:00:00,101972,337.45
17498,2018-12-31 19:00:00,140316,485.70
17499,2018-12-31 20:00:00,25525,62.35
